In [ ]:
import pandas as pd
import itertools
from sklearn.metrics import cohen_kappa_score

def get_cohens_kappa(df,vis_columns):
    cohenskappa_results = []

    # Pairwise comparisons
    for col1, col2 in itertools.combinations(vis_columns, 2):
        kappa = cohen_kappa_score(df[col1], df[col2])
        cohenskappa_results.append({
            'Column 1': col1,
            'Column 2': col2,
            'Cohen\'s Kappa': kappa
        })
    cohenskappa_results_df = pd.DataFrame(cohenskappa_results)
    return cohenskappa_results_df

vis_columns = ['Event_Name_dictionary','Event_Name_LLM_Events_no_evidence','Event_Name_LLM_Events_dict_evidence','Event_Name_LLM_Events_embedder_evidence','Event_Name_LLM_Events_all_evidence']
for file_name in ["only_event_mention_old","no_particular_event_mention","multi_event","unknown_event"]:
    disagreement_df = pd.read_pickle(f"../exports/disagreements/{file_name}.pkl")
    print(file_name)
    for col in vis_columns:
        disagreement_df[col] = disagreement_df[col].apply(lambda x: "_".join(tuple(set(x))) if type(x)==list else x)
    disagreement_df.to_excel(f"../exports/disagreements/{file_name}.xlsx")
    # Store results in a DataFrame
    cohenskappa_results_df = get_cohens_kappa(disagreement_df,vis_columns)
    
    cohenskappa_results_df.to_excel(f"../exports/disagreements/{file_name}_cohenskappa.xlsx")
    

only_event_mention_old
                                  Column 1  \
3                    Event_Name_dictionary   
2                    Event_Name_dictionary   
0                    Event_Name_dictionary   
1                    Event_Name_dictionary   
4        Event_Name_LLM_Events_no_evidence   
6        Event_Name_LLM_Events_no_evidence   
8      Event_Name_LLM_Events_dict_evidence   
7      Event_Name_LLM_Events_dict_evidence   
5        Event_Name_LLM_Events_no_evidence   
9  Event_Name_LLM_Events_embedder_evidence   

                                  Column 2  Cohen's Kappa  
3       Event_Name_LLM_Events_all_evidence       0.292256  
2  Event_Name_LLM_Events_embedder_evidence       0.337129  
0        Event_Name_LLM_Events_no_evidence       0.351093  
1      Event_Name_LLM_Events_dict_evidence       0.356631  
4      Event_Name_LLM_Events_dict_evidence       0.695249  
6       Event_Name_LLM_Events_all_evidence       0.732951  
8       Event_Name_LLM_Events_all_evidence       0

/home/asusaiyah/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/asusaiyah/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:897: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/asusaiyah/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/asusaiyah/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:897: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/asusaiyah/.local/l